## Extact planet data

In [1]:
# Importar las bibliotecas necesarias
import geemap
import ee
import geopandas as gpd
import json
import pandas as pd

In [2]:
# Autenticación y inicialización de GEE
ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [9]:
# Configurar el mapa
Map = geemap.Map(center=(12.20, -16.36), zoom=12)
Map

Map(center=[12.2, -16.36], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
Map.user_roi.getInfo()

In [5]:
# Asegurarnos de la creación de una geometría
geometry = ee.Geometry.Polygon([[[-16.459229, 12.239944],
   [-16.459229, 12.251184],
   [-16.426816, 12.251184],
   [-16.426816, 12.239944],
   [-16.459229, 12.239944]]])

In [10]:
# Cargar la colección de imágenes y aplicar filtros
collection = ee.ImageCollection("projects/ee-jesusc461/assets/Imagenes_planet_folder/Collection_planet") \
              .filterDate('2022-07-01', '2022-07-10') \
              .filterBounds(geometry)

# Función para recortar imágenes
def clip_image(image):
    return image.clip(geometry)

# Función para añadir días julianos como una banda
def add_julian_band(image):
    # Obtener la fecha de adquisición de la imagen
    date = ee.Date(image.get('system:time_start'))
    
    # Calcular los días julianos desde una fecha de referencia
    reference_date = ee.Date('2022-07-01')  # Ajusta según tu fecha de referencia
    julian_days = date.difference(reference_date, 'day')
    
    # Crear una imagen constante con los días julianos
    julian_band = ee.Image.constant(julian_days).rename('julian_days')
    
    # Añadir la banda de días julianos
    return image.addBands(julian_band)

# Función para añadir una banda con el año de la imagen
def add_year_band(image):
    # Obtener la fecha de adquisición de la imagen
    date = ee.Date(image.get('system:time_start'))
    
    # Extraer el año
    year = date.get('year')
    
    # Crear una imagen constante con el año
    year_band = ee.Image.constant(year).rename('year')
    
    # Añadir la banda del año
    return image.addBands(year_band)


# Aplicar la función de recorte a cada imagen en la colección
collection = collection.map(clip_image)
collection = collection.map(add_year_band)
collection = collection.map(add_julian_band)

# Función para añadir índices espectrales y convertir a float32
def addIndices(img):
    img = img.toFloat()  # Convertir todas las bandas a float32
    
    # Cálculo de índices existentes
    NDVI = img.normalizedDifference(['nir', 'red']).rename('NDVI')
    SR = img.select('nir').divide(img.select('red')).rename('SR')
    GCVI = img.expression('(NIR / GREEN) - 1', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GCVI')
    NDWI = img.expression('(GREEN - NIR) / (GREEN + NIR)', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('NDWI')
    VARI = img.expression('(GREEN - RED) / (GREEN + RED - BLUE)', {'BLUE': img.select('blue'), 'RED': img.select('red'), 'GREEN': img.select('green')}).rename('VARI')
    GRVI = img.expression('NIR / GREEN', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GRVI')
    GNDVI = img.normalizedDifference(['nir', 'green']).rename('GNDVI')
    SAVI = img.expression('(NIR - RED) / ((NIR + RED + 0.5) * 1.5)', {'NIR': img.select('nir'), 'RED': img.select('red')}).rename('SAVI')
    # Índices adicionales que utilizan el borde rojo
    NDRE = img.expression('(NIR - RED_EDGE) / (NIR + RED_EDGE)', {'NIR': img.select('nir'), 'RED_EDGE': img.select('rededge')}).rename('NDRE')
    NDVIre = img.expression('(NIR - RED_EDGE) / (NIR + RED_EDGE)', {'NIR': img.select('nir'), 'RED_EDGE': img.select('rededge')}).rename('NDVIre')
    REI = img.expression('(RED_EDGE - RED) / (RED_EDGE + RED)', {'RED_EDGE': img.select('rededge'), 'RED': img.select('red')}).rename('REI')
    
    # Añadiendo todos los índices calculados a la imagen
    return img.addBands([NDVI, SR, GCVI, NDWI, VARI, GNDVI, GRVI, SAVI, NDRE, NDVIre, REI])

# Aplicar la función de índices a la colección
collection_with_indices = collection.map(addIndices)

# Agrupar por fecha y obtener el promedio de las imágenes (tiles) por fecha
def mosaic_by_date(date, collection):
    date_str = ee.Date(date).format('YYYY-MM-dd')
    date_collection = collection.filter(ee.Filter.date(ee.Date(date), ee.Date(date).advance(1, 'day')))
    mosaic = date_collection.mosaic().set('system:time_start', date).toFloat()  # Promediar y convertir a float32
    return mosaic

# Obtener las fechas únicas en la colección
dates = ee.List(collection_with_indices.aggregate_array('system:time_start')).distinct()

# Crear una nueva colección con las imágenes promediadas por fecha
mosaicked_collection = ee.ImageCollection(dates.map(lambda date: mosaic_by_date(date, collection_with_indices)))

# Aplicar la función a la colección de imágenes
ps_1 = collection.map(addIndices)

# Crear un composite de las imágenes
composite = ps_1.mean().float()

# Añadir la capa al mapa
Map.addLayer(composite, {'bands': ['nir', 'red', 'green'], 'min': 201, 'max': 2464}, 'RGB')
Map.addLayer(geometry)


In [11]:
# Exportar cada imagen de la colección a Google Drive
def export_image(image):
    date = image.date().format('YYYY-MM-dd').getInfo()
    description = f'Planet_Image_{date}'
    
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        scale=3,
        region=geometry,
        folder='GEE_Exports',
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    task.start()

# Convertir la colección mosaicked a una lista y exportar cada imagen
image_list = mosaicked_collection.toList(mosaicked_collection.size())
for i in range(image_list.size().getInfo()):
    image = ee.Image(image_list.get(i))
    export_image(image)